In [1]:
import io
import os
import time
import yaml
import warnings
import pandas as pd
import json
import hashlib
import csv
from influxdb_client import InfluxDBClient
from influxdb_client.client.warnings import MissingPivotFunction
from dotenv import load_dotenv

import numpy as np
from functools import partial
import glob

# Suppress specific warnings
warnings.simplefilter("ignore", MissingPivotFunction)

# Load environment variables
load_dotenv()

True

In [2]:
# Configuration
GET_DATA_FROM_FETCH = True 
GET_DATA_FROM_INFLUX = False
INFLUX_BUCKET = "aruba"
OUTPUT_FILE = 'ap_data_all_floors.csv'

In [3]:
# InfluxDB query
INFLUX_QUERY = '''from(bucket: "aruba")
  |> range(start: 2024-04-15T07:00:00Z, stop: 2024-07-30T12:00:00Z)
  |> filter(fn: (r) => r["_measurement"] == "adjacent-ap")
  |> filter(fn: (r) => r["building"] == "D1")
  |> filter(fn: (r) => r["ap-type"] == "valid")
  |> filter(fn: (r) => r["_field"] == "curr-rssi" or r["_field"] == "crawling-duration (ms)")
  |> aggregateWindow(every: 15m, fn: mean, createEmpty: false)
  |> yield(name: "mean")'''

In [4]:
def get_influx_data(query):
    """Fetch data from InfluxDB."""
    token = os.environ.get("INFLUXDB_TOKEN")
    INFLUX_URL = os.environ.get("INFLUX_URL")
    INFLUX_ORG = os.environ.get("INFLUX_ORG")
    if not token:
        raise ValueError("INFLUXDB_TOKEN not found in environment variables")

    client = InfluxDBClient(url=INFLUX_URL, token=token, org=INFLUX_ORG)
    query_api = client.query_api()
    
    data = query_api.query_data_frame(org=INFLUX_ORG, query=query)
    print(f"Data fetched: {data.shape}")
    return data

In [5]:
def compute_sha256(bss):
    return hashlib.sha256(bss.encode()).hexdigest()

def process_data(data, floor_filters):
    """Process the raw data with optimizations."""
    columns = ['_time', 'essid', 'adjacent-ap-bssid', 'band', 'monitoring-ap', '_value']
    data = data[columns]
    
    # Use boolean indexing for filtering
    mask = (data['band'] == '2.4GHz') & \
           (data['monitoring-ap'].notna()) & \
           (data['monitoring-ap'].str.contains(floor_filters, regex=True))
    data = data[mask]
    
    mask_64 = data["adjacent-ap-bssid"].str.len() == 64
    data.loc[~mask_64, "adjacent-ap-bssid"] = data.loc[~mask_64, "adjacent-ap-bssid"].apply(compute_sha256)
    
    # Pivot the data
    pivot_table = pd.pivot_table(data, 
                                 values='_value', 
                                 index=['_time', 'essid', 'adjacent-ap-bssid', 'band'],
                                 columns='monitoring-ap')
    
    result = pivot_table.reset_index()
    result.columns = ['_time', 'essid', 'bssid', 'band'] + ['rssi_' + str(col) for col in pivot_table.columns]
    
    return result

In [6]:
def compute_sha256(value):
    """Compute SHA256 hash of a string."""
    return hashlib.sha256(value.encode()).hexdigest()


def translate_bssids(df, floor_filters):
    with open('../data/ap_bss_table.json', 'r') as file:
        bss_data = json.load(file)

    bss_df = pd.DataFrame(bss_data["Aruba AP BSS Table"])
    bss_df['bssid'] = bss_df['bss'].apply(compute_sha256)

    merged_df = pd.merge(df, bss_df[['bssid', 'ap name']], on='bssid', how='left')
    merged_df.rename(columns={'ap name': 'ap_name'}, inplace=True)
    merged_df = merged_df[merged_df['ap_name'].notna()]
    merged_df = merged_df[merged_df['ap_name'].str.contains(floor_filters, regex=True)]

    return merged_df 


In [7]:
def read_coordinate_file(file_path):
    """Read AP coordinate data from a CSV file."""
    with open(file_path, 'r') as file:
        return {row[0]: (float(row[1]), float(row[2]), float(row[3])) for row in csv.reader(file)}

def add_coordinates(df, ap_data):
    """Add x, y, and z coordinates to the dataframe."""
    df[['x', 'y', 'z']] = df['ap_name'].map(ap_data).apply(pd.Series)
    return df


In [8]:
floor_files = sorted(glob.glob('../data/[2-9]f.csv') + glob.glob('../data/1[0-3]f.csv'))
floor_numbers = [file.split('/')[-1].split('f')[0] for file in floor_files]
floor_filters = '|'.join([f'D1_{floor}F' for floor in floor_numbers])

ap_data = {}
for file in floor_files:
    ap_data.update(read_coordinate_file(file))

In [9]:
if GET_DATA_FROM_FETCH:
    # fetch using web api and js
    with open('../data/aruba_07_15.json', 'r') as file:
        data = file.read()

    # Use StringIO to create a file-like object from the string
    data_io = io.StringIO(data)

    df = pd.read_csv(data_io, 
                    comment='#',
                    skiprows=3,  
                    parse_dates=['_start', '_stop', '_time'], 
                    na_values=[''],  
                    #nrows=50000,
                    )
else:
    if GET_DATA_FROM_INFLUX:
        df = get_influx_data(INFLUX_QUERY)    
        filename = f"data/aruba_data_{int(time.time())}.csv"
        df.to_csv(filename, index=False)
    else:
        df = pd.read_csv('data/aruba_2024-07-15T07:00:00Z_stop_2024-07-16T12:00:00Z')

/tmp/ipykernel_9106/3931779889.py:9: DtypeWarning: Columns (1,2,6,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_io,


In [11]:
processed_data = process_data(df, floor_filters)
clean_df = translate_bssids(processed_data, floor_filters)
final_df = add_coordinates(clean_df, ap_data)


In [12]:
final_df.shape

(70452, 70)

In [13]:
# Save the final dataframe
final_df.to_csv(OUTPUT_FILE, index=False)
print(f"Data saved to {OUTPUT_FILE}")

# Print some statistics
print(f"Number of different APs: {final_df['ap_name'].nunique()}")
print(f"Date range: {final_df['_time'].min()} to {final_df['_time'].max()}")
print(f"Total records: {len(final_df)}")
print(f"Z-coordinate range: {final_df['z'].min()} to {final_df['z'].max()}")
print(f"Floors included: {', '.join(floor_numbers)}")

Data saved to ap_data_all_floors.csv
Number of different APs: 135
Date range: 2024-07-14T03:00:00Z to 2024-07-15T02:48:00Z
Total records: 70452
Z-coordinate range: 32.4042 to 195.111
Floors included: 10, 11, 12, 13, 2, 3, 4, 5, 6, 7, 8, 9
